In [116]:
#load the required libraries
import mysql.connector as my
import pandas as pd
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from pandasql import sqldf
from ggplot import *

pysqldf = lambda q: sqldf(q, globals())

###Connect to Analytics Snapshot and extract distinct Users who have made a Booking

In [66]:
con = my.connect(host = 'analytics-db.housetrip.com', port = 3306, user = 'jbarnett', db = 'housetrip_production', passwd = 'xxxxxx')

In [123]:
guest_query = '''
    SELECT
        b.user_id,
        min(b.accepted_date) as first_booking,
        g.first_destination,
        b.user_country
    FROM dw_bookings b
    LEFT JOIN dw_guests g on g.user_id = b.user_id
    WHERE b.status = 'accepted'
    GROUP BY b.user_id, g.first_destination, b.user_country
    '''

guests = pd.read_sql(guest_query, con = con)

In [130]:
#check outputs
print 'Data Frame of ' + str(guests.shape[0]) + ' rows and ' + str(guests.shape[1]) + ' columns'
guests.head()

Data Frame of 474532 rows and 4 columns


,user_id,first_booking,first_destination,user_country
0,74,2009-09-09,Munich,Germany
1,75,2009-10-11,Lisbon,United Kingdom
2,76,2009-12-10,Madrid,Switzerland
3,78,2013-02-11,Valletta,Germany
4,80,2009-09-10,Ljubljana,United Kingdom


###Generate table of months

In [4]:
start = date(2009,9,1)
end = date(2015,6,1)

month = []

while start < end:
    month.append(start)
    start += relativedelta(months=1)
    
months = pd.DataFrame(month)
months.columns = ['month']
months['month'] = pd.to_datetime(months['month']) #convert to date datatype 
months.head()

,month
0,2009-09-01
1,2009-10-01
2,2009-11-01
3,2009-12-01
4,2010-01-01


###Loop over Users to create user_month data frame 

In [ ]:
#create empty master data frame to append each users months to
master_df = pd.DataFrame(columns = ['month', 'user_id'])
master_df['month'] = pd.to_datetime(master_df['month'])
master_df['user_id'] = master_df['user_id'].astype(int)

In [ ]:
#### The following loops over the guests and produces the months post their inital booking 
#### which are appended to the master data frame along with the user-id.

for i in range(0, len(guests)):

    the_guest = guests.iloc[i, 0]
    first_booking_month = guests.iloc[i, 1]
    the_months = months[(months['month'] > first_booking_month)]
    user_df = pd.DataFrame(the_months)
    user_df['user_id'] = the_guest
        
    master_df = pd.concat([master_df, user_df])
        
    if the_guest % 1000 == 0:
        print(the_guest)


In [131]:
#Check Outputs
print 'Data Frame of ' + str(master_df.shape[0]) + ' rows and ' + str(master_df.shape[1]) + ' columns'

Data Frame of 9792952 rows and 3 columns


In [82]:
#save as csv in case of emergencies (long runtime)
master_df.to_csv('user_month.csv', sep = ',')

###Get Repeat Data

In [106]:
repeat_query = '''
SELECT
    DATE_ADD(accepted_date, INTERVAL - DAY(accepted_date) + 1 DAY) as month,
    user_id
FROM dw_bookings
WHERE 
    repeat_type = 'repeat'
    AND accepted_date < '2015-06-01'
GROUP BY 
    DATE_ADD(accepted_date, INTERVAL - DAY(accepted_date) + 1 DAY),
    user_id
    '''

repeats = pd.read_sql(repeat_query, con = con)
repeats['month'] = pd.to_datetime(repeats['month'])
repeats['user_id'] = repeats['user_id'].astype(int)

master_df['user_id'] = master_df['user_id'].astype(int)

###Add column of 1 or 0 to user_month based on whether they repeated in that month.

In [124]:
repeat_update_query = '''
SELECT
m.month, 
m.user_id, 
CASE WHEN r.user_id IS NOT NULL THEN 1 ELSE 0 END as repeater
FROM master_df m 
LEFT JOIN repeats r 
    ON m.month = r.month 
    AND m.user_id = r.user_id
'''
df = pysqldf(repeat_update_query)

In [125]:
df.head()

,month,user_id,repeater
0,2009-10-01,74,0
1,2009-11-01,74,1
2,2009-12-01,74,0
3,2010-01-01,74,0
4,2010-02-01,74,0


###The main structure is in place, now to add some features.......
####Features to add

1. user country (aka fr, other_eu, row)
2. initial destination
3. months post initital booking
4. post initial booking checkin
5. nps
6. months since last search (post booking acceptance).
7. prior repeater
8. google spend in that market (standardised)
9. was TV on air in that market (binary)
10. original marketing attribution
11. 

###Add user country, initial destination and original booking date

In [136]:
df = pd.merge(df, guests, on = 'user_id')
df.head()

,month,user_id,repeater,first_booking,first_destination,user_country
0,2009-10-01,74,0,2009-09-09,Munich,Germany
1,2009-11-01,74,1,2009-09-09,Munich,Germany
2,2009-12-01,74,0,2009-09-09,Munich,Germany
3,2010-01-01,74,0,2009-09-09,Munich,Germany
4,2010-02-01,74,0,2009-09-09,Munich,Germany


### Create list of major countries

In [138]:
country_query = '''
    SELECT
        user_country,
        count(1) as bookings
    FROM dw_bookings
    GROUP BY user_country
    ORDER BY bookings DESC
'''
countries = pd.read_sql(country_query, con = con)

In [ ]:
#add column that assesses whether country has had > 1000 bookings, any that haven't are called row (rest of world)

def maj_market(row):
    if row['bookings'] >= 1000:
        return row['user_country']
    else:
        return 'row'
    
countries['main_market'] = countries.apply(maj_market, axis=1)

In [141]:
countries = countries.drop('bookings', 1)

In [144]:
df = pd.merge(df, countries, on = 'user_country')

###Read in adwords data

In [133]:
adwords = pd.read_csv('adwords.csv', sep = '\t')
adwords.columns = ['account', 'month', 'country', 'clicks', 'cost']
#adwords.groupby('country').sum().sort('cost', ascending = False).head()

In [145]:
df.head()

,month,user_id,repeater,first_booking,first_destination,user_country,main_market
0,2009-10-01,74,0,2009-09-09,Munich,Germany,Germany
1,2009-11-01,74,1,2009-09-09,Munich,Germany,Germany
2,2009-12-01,74,0,2009-09-09,Munich,Germany,Germany
3,2010-01-01,74,0,2009-09-09,Munich,Germany,Germany
4,2010-02-01,74,0,2009-09-09,Munich,Germany,Germany
